In [66]:
from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import numpy as np
import pandas as pd
import os, sys
from matplotlib import pyplot as plt
%matplotlib inline
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)
#print("Project root:", project_root)




cpu


In [67]:
filename="500ktest_dataset_lar.csv"
script_dir = os.path.dirname('__file__')
data_path = os.path.join(script_dir, '..', 'datasets', filename)
data_path = os.path.abspath(data_path)
df=pd.read_csv(data_path)

/var/folders/b0/ymr48tf963d9_3y2jfmn5wfw0000gn/T/ipykernel_3318/697764039.py:5: DtypeWarning: Columns (23,24,25,27,28,29,30,31,32,37,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(data_path)


In [68]:
df.head()

,Unnamed: 0,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,origination_charges,discount_points,lender_credits,loan_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,co_applicant_ethnicity_1,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,co_applicant_race_1,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,denial_reason_1,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,98395,2024,549300ZX4OGRPOOEH505,28140,KS,20209.0,2.020904e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Hispanic or Latino,White,Female,1,0,2,1,31,1,2,2,2,125000,90.0,3.18,-4.925,2,984.0,0.0,NaN,NaN,360.0,12.0,2,2,2,2,135000.0,1,1,3,5,1,NaN,37,2,10,1.0,5.0,2,4,5.0,8.0,2,4,2,5,2,4,25-34,9999,No,NaN,1,1,6,10,1125,77.69,103000,0.0,197,398,43
1,55868,2024,5493000NYUJT9UC6G261,45294,FL,12101.0,1.210103e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Male,1,1,2,1,32,1,2,2,2,205000,80.0,7.49,0.947,2,7515.0,4865.0,3120.0,NaN,360.0,NaN,2,2,2,2,265000.0,1,1,3,5,1,76.0,30%-<36%,3,10,2.0,5.0,2,4,5.0,8.0,2,4,1,5,2,4,25-34,9999,No,NaN,1,1,2,10,5067,39.73,88800,155.0,1401,1824,0
2,90858,2024,5493006MW6O2CE88BD43,39820,CA,6089.0,6.089011e+09,C,Conventional:Subordinate Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,3,0,2,1,2,2,2,2,2,25000,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,2,NaN,1,1,3,5,1,47.0,50%-60%,9,10,2.0,5.0,1,4,5.0,8.0,1,4,2,5,1,4,65-74,9999,Yes,NaN,1,1,6,1,3467,23.74,89100,139.0,948,1242,34
3,12143,2024,549300157R33LDICP344,99999,OH,39021.0,3.902101e+10,C,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Female,1,71,2,2,1,1,2,2,2,215000,96.50000,6.500,1.310,2,7630.81,2125.00,NaN,527.76,360,NaN,2,2,2,2,215000,1,1,3,5,1,67.0,46,1,10,2.0,5.0,2,4,5.0,8.0,2,4,2,5,2,4,25-34,9999,No,NaN,1,1,3,10,3714,16.32,81600,105.0,993,1474,59
4,4785,2024,ZF85QS7OXKPBG52R7N18,33340,WI,55079.0,5.507902e+10,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,White,Male,4,0,2,1,1,1,2,2,2,205000,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,360,NaN,2,2,2,2,NaN,1,1,3,5,1,60.0,NaN,9,9,2.0,5.0,2,4,5.0,8.0,2,4,1,5,2,4,25-34,9999,No,NaN,1,1,1,10,4145,22.46,102100,91.0,1337,1944,74


In [56]:
#Pre-Processing.
target='action_taken'
nan_counts = df.isna().sum()
nan_counts = nan_counts[nan_counts > 0].sort_values(ascending=False)
#print("Columns with missing values:\n", nan_counts)
print(list(df.columns))

['Unnamed: 0', 'activity_year', 'lei', 'derived_msa_md', 'state_code', 'county_code', 'census_tract', 'conforming_loan_limit', 'derived_loan_product_type', 'derived_dwelling_category', 'derived_ethnicity', 'derived_race', 'derived_sex', 'action_taken', 'purchaser_type', 'preapproval', 'loan_type', 'loan_purpose', 'lien_status', 'reverse_mortgage', 'open_end_line_of_credit', 'business_or_commercial_purpose', 'loan_amount', 'combined_loan_to_value_ratio', 'interest_rate', 'rate_spread', 'hoepa_status', 'total_loan_costs', 'origination_charges', 'discount_points', 'lender_credits', 'loan_term', 'intro_rate_period', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'property_value', 'construction_method', 'occupancy_type', 'manufactured_home_secured_property_type', 'manufactured_home_land_property_interest', 'total_units', 'income', 'debt_to_income_ratio', 'applicant_credit_score_type', 'co_applicant_credit_score_type', 'applicant_ethnicit

In [ ]:
pd.set_option('display.max_columns', 100)  # or however many you want
pd.set_option('display.max_rows', 200)
#pd.crosstab(df['intro_rate_period'], df['derived_loan_product_type'])
feature='applicant_race_1'
print("Number of NaNs:", df[feature].isna().sum())
print(df[feature].value_counts().head(200))

#to_process
census_features=['tract_population', 'tract_minority_population_percent',
       'ffiec_msa_md_median_family_income', 'tract_to_msa_income_percentage',
       'tract_owner_occupied_units', 'tract_one_to_four_family_homes',
       'tract_median_age_of_housing_units']

numericwithranges=['debt_to_income_ratio']
pd_to_numeric=['lender_credits', 'income', 'loan_amount']
cols_with_exempt_numeric_to_process=['intro_rate_period', 'discount_points', 'total_loan_costs', 'origination_charges', 'rate_spread', 'interest_rate', 'combined_loan_to_value_ratio', 'property_value',
                                     'loan_term', ]
Boolean_with_exempt=['reverse_mortgage', 'open_end_line_of_credit', 'business_or_commercial_purpose', 'hoepa_status', 'negative_amortization', 'interest_only_payment', 'balloon_payment', 'other_nonamortizing_features', 'manufactured_home_land_property_interest']
stringtobool=['co_applicant_age_above_62', 'applicant_age_above_62']
cat_to_embed=['lei', 'derived_msa_md', 'census_tract', 'county_code', 'state_code', 'applicant_race_1', 'applicant_ethnicity_1', 'co_applicant_ethnicity_1', 'co_applicant_race_1']
cat_to_one_hot=['total_units', 'occupancy_type', 'manufactured_home_secured_property_type', 'conforming_loan_limit','derived_loan_product_type', 'derived_dwelling_category', 'derived_sex', 'purchaser_type', 'action_taken', 'loan_type', 'loan_purpose', 'lien_status', 
                'applicant_age', 'co_applicant_age', 'initially_payable_to_institution', 'denial_reason_1']

unused_feat = ['aus_1', 'submission_of_application', 'co_applicant_sex', 'applicant_sex_observed', 'co_applicant_sex_observed', 'applicant_race_observed', 'co_applicant_race_observed', 'Set', 'Unnamed: 0', 'activity_year', 'derived_ethnicity', 'derived_race', 'applicant_credit_score_type', 'co_applicant_credit_score_type', 'applicant_ethnicity_observed', 'co_applicant_ethnicity_observed', 'applicant_sex']
boolean=['preapproval', 'construction_method']
numeric=[]
categorical=[]


Number of NaNs: 126
applicant_race_1
5.0     288420
6.0      73745
7.0      58216
3.0      42857
2.0      25248
1.0       4984
21.0      2660
4.0        955
27.0       744
23.0       554
22.0       448
26.0       294
44.0       253
25.0       217
24.0        78
42.0        42
41.0        33
43.0        15
Name: count, dtype: int64


In [ ]:
from utilities.datatransformation import process_numeric_with_exempt


In [ ]:
#Encode Categoricals
from sklearn.preprocessing import LabelEncoder
categorical_columns=[]
categorical_dims={}

unused_feat = ['Set', 'sig_id'] # Let's not use splitting sets and sig_id
features = [ col for col in df.columns if col not in unused_feat] 
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [ ]:
#Train Test Split
if "trainingset" not in df.columns:
    df["trainingset"] = np.random.choice(["train", "valid", "test"], p =[.8, .1, .1], size=(df.shape[0],))
train_indices = df[df['trainingset']=="train"].index
valid_indices = df[df['trainingset']=="valid"].index
test_indices = df[df['trainingset']=="test"].index
X_train = df[features].values[train_indices]
y_train = df[target].values[train_indices]

X_valid = df[features].values[valid_indices]
y_valid = df[target].values[valid_indices]

X_test = df[features].values[test_indices]
y_test = df[target].values[test_indices]

In [ ]:
tabnet_classifier_params={
                'n_d': 64,
                'n_a': 64,
                'n_steps': 5,
                
                "cat_idxs":cat_idxs,
                "cat_dims":cat_dims,
                "cat_emb_dim":2,
                "optimizer_fn":torch.optim.Adam,
                "optimizer_params":dict(lr=2e-2),
                "scheduler_params":{"step_size":50, # how to use learning rate scheduler
                                 "gamma":0.9},
                "scheduler_fn":torch.optim.lr_scheduler.StepLR,
                "mask_type":'entmax', # "sparsemax"
                "grouped_features" : grouped_features
                }

model=TabNetClassifier(**tabnet_classifier_params)
model.fit(
    X_train=X_train, y_train=y_train,
    X_valid=X_valid, y_valid=y_valid,
    max_epochs=1000 ,
    patience=50, # please be patient ^^
    batch_size=1024,
    virtual_batch_size=128,
    num_workers=1,
    drop_last=False,
)



In [ ]:
plt.plot(model.history['loss'])
plt.plot(model.history['train_auc'])
plt.plot(model.history['valid_auc'])
plt.plot(model.history['lr'])
# plot accuracy
plt.plot(model.history['train_accuracy'])
plt.plot(model.history['valid_accuracy'])

In [ ]:
#Prediction:
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test)
test_acc = accuracy_score(y_pred=y_pred, y_true=y_test)
print(f"FINAL TEST SCORE FOR {filename} : {test_acc}")
saved=model.save_model('test_model')
#load by doing model.load_model(saved)

In [ ]:
#Explanations
model.feature_importances_